# Collect NLDAS Forcing Data

This notebook demonstrates the process of collecting NLDAS forcing data, regridding these data onto an existing WRF-Hydro (domain collected in the previous notebook).

## 1. Objective

This notebook illustrates the process of collecting and regridding NLDAS data for a WRF-Hydro domain. This include:

1. Subsetting NLDAS data from Earthdata.nasa.gov 
2. Downloading NLDAS data
3. Regridding NLDAS using the ESMF regridding scripts


In [ ]:
pip install python3-wget

In [ ]:
import os
import sys
import wget
import time
import glob
import shutil
import tarfile
import getpass 
import urllib.parse as p
import multiprocessing as mp
from urllib.request import urlopen
from subprocess import Popen, PIPE, STDOUT, check_output, CalledProcessError

Make sure that the NCAR Command Language is installed. Detailed instructions can be found [here](https://www.ncl.ucar.edu/Download/conda.shtml). This following cell will install `ncl` into your conda environment if it doesn't already exist.

In [ ]:
# make sure we invoke the NCL that is installed in our conda environment
ncl_exec = os.path.join(os.path.dirname(sys.executable), 'ncl')
try:
    output = check_output([ncl_exec, '-V'])
    print('NCL is already installed :)')
except Exception as e:
    !conda install -y ncl

## 1. Subset NLDAS data from Earthdata

See this [link](https://hydro1.gesdisc.eosdis.nasa.gov/data/NLDAS/README.NLDAS2.pdf) for details about primary (FORA) and secondary (FORB) NLDAS forcing datasets. The following steps are completed in the https://urs.earthdata.nasa.gov/ website:

1. Create Earthdata account: [link](https://urs.earthdata.nasa.gov/users/new)
2. Link GES DISC with your earthdata account: [link](https://disc.gsfc.nasa.gov/earthdata-login)
3. Select NLDAS2 data via: [https://disc.gsfc.nasa.gov/datasets/NLDAS_FORA0125_H_V002/summary?keywords=NLDAS](https://disc.gsfc.nasa.gov/datasets/NLDAS_FORA0125_H_V002/summary?keywords=NLDAS) and subset the data to a reasonable size and date range using the `Simple Subset Wizard`.  
5. Specify the date range   
6. Use the 'Spatial Bounding Box' to trace a rough area of where your watershed exists. (This DOES NOT need to be exact!)
7. Choose the following variables to extract:
    - 10-m above ground Meridional wind speed
    - 10-m above ground Zonal wind speed 
    - 2-m above ground Specific humidity
    - 2-m above ground Temperature 
    - LW radiation flux downwards
    - Precipitation hourly total
    - Surface pressure
    - SW radiation flux downwards
8. Select file type `GRIB` from the dropdown menue
9. Click "Subset Selected Data Sets" 
10. Click "View Subset Results"
11. Above the list of files, select the "list of URLs"  
12. Copy the URL in the address bar at the top of the page with the URL list

Specify the URL for downloading the subsetted NLDAS data, from the previous step, below. For example, https://disc.gsfc.nasa.gov/SSW/WWW-TMP/SSW_download_2019-06-19T18:32:48_60480_NukeoWYr.inp. This URL will be used to write the link addresses for each NLDAS dataset to a local file.  

Run the following code block and you will be prompted to paste in your URL for the Forcing from the last step.

In [ ]:
file_urls = input("Enter EarthData Download URL: ")

# read all of the urls defined in the link above
input_dir = 'input_files'
if os.path.exists(input_dir):
    while 1:
        res = input(f"Directory '{input_dir}' already exists. Do you wish to remove it [Y/N]?")
        if res.lower() == 'y':
            shutil.rmtree(input_dir)
            os.mkdir(input_dir)
            break
        elif res.lower() == 'n':
            break
        else:
            print('\nInvalid input. Please answer either Y or N')
else:
    os.mkdir(input_dir)

# write these urls to a local file
print(f'Writing NLDAS urls to {input_dir}/urls.txt')
f = urlopen(file_urls)
urls = f.read().decode('utf-8')
with open('input_files/urls.txt', 'w') as f:
    f.write(urls)        

Download the files listed in `input_files/urls.txt` using `wget`. This may take several minutes depending on how many files you're downloading. This following command can be executed in a separate terminal window to watch the files as they're downloaded:

```
$watch "ls -l <path to data>/input_files | wc -l"
```

Define a function to download all urls in our file 

In [ ]:
# define a function to download nwm results
def get_nldas(q, iolock, out_q, cnt):
    while True:
        user, pwd, url, total = q.get()        
        if url is None:
            break
        outfile = os.path.join('input_files', url.split('LABEL=')[-1].split('&')[0])
        !wget --quiet \
              --auth-no-challenge=on \
              --content-disposition \
              --user {user} \
              --password {pwd} \
              -O {outfile} \
              "{url}"
        with iolock:
            cnt.value += 1
            percent = cnt.value / total * 100
            print(f'\r[{cnt.value} of {total}] files downloaded -- {percent:.2f}% complete', end=10*' ')

Invoke this function in parallel to speed up the download.

In [ ]:
user = input("Earthdata User: ")
password = getpass.getpass("Earthdata Password: ")

NCORE = 4
in_q = mp.Queue(maxsize=NCORE)
out_q = mp.Queue()
cnt = mp.Value('i', 0)
iolock = mp.Lock()

pool = mp.Pool(NCORE, initializer=get_nldas, 
               initargs=(in_q, iolock, out_q, cnt))

with open('input_files/urls.txt', 'r') as f:
    lines = f.readlines()
    total = len(lines)
    for url in lines:
        time.sleep(.1)
        in_q.put((user, password, url, total))  # blocks until q below its max size
for _ in range(NCORE):  # tell workers we're done
    in_q.put((None, None, None, None))
pool.close()
pool.join()

Preview the data that we downloaded. 

In [ ]:
%%bash

du -h input_files/*

The data that we've downloaded are NLDAS NetCDF files that have been subsetted through time, space, and variable. These data still need to be regridded to the WRF-Hydro domain that was collected in the previous notebook.

Create regridding 'weight' files required by the ESMF regridders. The weight files are netCDF files which specify interpolation weights between the source coordinate data grids (src) and destination coordinate data (dst) grids. The weight file is generated by running the `NLDAS2WRFHydro_generate_weights.ncl` script. We'll need to provide the source and destination grid filenames as arguments to the script, for example:

```
$ ncl interp_opt="bilinear"
    srcGridName=<NLDAS NetCDF File>
    dstGridName=<WRF-Hydro geo_em.d01.nc>
    NLDAS2WRFHydro_generate_weights.ncl
```

This will create the following files:

```
DAS2WRFHydro_weight_bilinear.nc  
PET0.RegridWeightGen.Log  
SCRIP_NLDAS_bilinear.nc  
SCRIP_WRFHydro_bilinear.nc  
```

Collect the regridding scripts from NCAR. These can be found on the WRF-Hydro website: https://ral.ucar.edu/projects/wrf_hydro/regridding-scripts 

In [ ]:
# collect the regridding scripts

# download the subset archive
print('Downloading ESMF regridding scripts')
archive_name = wget.download('https://ral.ucar.edu/sites/default/files/public/ESMFregrid_NLDAS.tar_.gz')

# untar the archive
print('Extracting archive contents')
tar = tarfile.open(archive_name)
tar.extractall()
tar.close()

# move the domain files into the DOMAIN directory
print('Organizing domain data')
extracted_folder = 'NLDAS'
for f in glob.glob(os.path.join(extracted_folder, '*.ncl')):
    shutil.move(f, os.getcwd())

print('Cleaning up')
os.remove(archive_name)
shutil.rmtree(extracted_folder)

Create the regridding weight file using the first NLDAS file in the `input_files` directory.

In [ ]:
%%bash -s "$ncl_exec"

FILE=`find input_files -name 'NLDAS*' | head -1`

$1 'interp_opt="bilinear"' \
'srcGridName="'$FILE'"' \
'dstGridName="DOMAIN/geo_em.d01.nc"' \
NLDAS2WRFHydro_generate_weights.ncl


Regrid all of the NLDAS files in the `input_files` directory using the NLDAS2WRFHydro_regrid.ncl script. This script takes NLDAS data and a weight file as inputs and outputs regridded data.

In [ ]:
%%bash -s "$ncl_exec" 

$1 'srcFileName="NLDAS*"' \
'dstGridName="DOMAIN/geo_em.d01.nc"' \
NLDAS2WRFHydro_regrid.ncl > regrid.log 2> regrid.err


Clean the our directory by (1) removing the raw NLDAS data that we downloaded from EarthData and (2) rename the default `output_files` directory to `FORCING`.

In [ ]:
forcing_dir = 'FORCING'
if os.path.exists(forcing_dir):
    while 1:
        res = input("Directory 'FORCING' already exists. Do you wish to remove it [Y/N]?")
        if res.lower() == 'y':
            shutil.rmtree(forcing_dir)
            os.mkdir(forcing_dir)
            break
        elif res.lower() == 'n':
            break
        else:
            print('\nInvalid input. Please answer either Y or N')
else:
    os.mkdir(forcing_dir)

# move the regridded files into the FORCING directory
print('Organizing FORCING data')
output_dir = 'output_files'
for f in glob.glob(os.path.join(output_dir, '*')):
    shutil.move(f, forcing_dir)
    
print('Cleaning up')
shutil.rmtree(output_dir)
shutil.rmtree('input_files')
regrid_files = []
regrid_files.extend(glob.glob('NLDAS2WRFHydro*'))
regrid_files.extend(glob.glob('SCRIP*'))
regrid_files.extend(glob.glob('PET0*'))
for f in regrid_files:
    os.remove(f)
    
!du -h FORCING/*
